# Explore here

In [57]:
# Import required libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns

In [58]:
# Load data in pandas dataframes
movies_df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv', sep=',')
credits_df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv', sep=',')

In [59]:
# Gain first insights into the data
print('🔹 Movie data')
print(movies_df.head())
print(movies_df.info())
print(movies_df.describe())

print('🔹 Credits data')
print(credits_df.head())
print(credits_df.info())
print(credits_df.describe())

🔹 Movie data
      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"i

In [60]:
# Create SQL database (in sqlite) and create new tables from dataframes

engine = create_engine("sqlite:///my_database.db", echo=True)

movies_df.to_sql('movies', con=engine, if_exists='replace', index=False)
credits_df.to_sql('credits', con=engine, if_exists='replace', index=False)

query = """
SELECT movies.title, movies.overview, movies.genres, movies.keywords, credits.movie_id, credits.cast, credits.crew  FROM movies
INNER JOIN credits ON movies.title = credits.title
"""

joined_table = pd.read_sql(query, con=engine)
print(joined_table.head())

# joined_table.to_sql('joined_data', con=engine, if_exists='replace', index=False)
# print('🔹 Visualize joined table')
# print(pd.read_sql('SELECT * FROM joined_data LIMIT 5', con=engine))

2025-02-24 14:35:33,613 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-24 14:35:33,619 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("movies")
2025-02-24 14:35:33,619 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-24 14:35:33,626 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("movies")
2025-02-24 14:35:33,627 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-24 14:35:33,628 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-02-24 14:35:33,629 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-24 14:35:33,630 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-02-24 14:35:33,630 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-24 14:35:33,631 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("movies")
2025-02-24 14:35:33,631 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-02-24 14:3

In [61]:
# Handle JSON data
import json

json_cols = ['genres', 'keywords', 'cast', 'crew', 'overview']

for col in json_cols:
    if col == 'cast_id':
        joined_table[col] = joined_table[col].apply(lambda x: [item['name'] for item in json.loads(x)[:3]])
    elif col == 'crew':
        joined_table[col] = joined_table[col].apply(lambda x: [item['name'] for item in json.loads(x)[:3] if item['job'] == 'Director'])
    elif col == 'overview':
        joined_table[col] = joined_table[col].apply(lambda x: [str(x)])
    else:
        joined_table[col] = joined_table[col].apply(lambda x: [item['name'] for item in json.loads(x)])

columns_to_modify = ['genres', 'cast', 'crew', 'keywords']

for col in columns_to_modify:
    joined_table[col] = joined_table[col].apply(lambda x: [i.replace(" ", "") for i in x])

joined_table["tags"] = joined_table["overview"] + joined_table["genres"] + joined_table["keywords"] + joined_table["cast"] + joined_table["crew"]
joined_table["tags"] = joined_table["tags"].apply(lambda x: " ".join(x))

print(joined_table.head())

                                      title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                            overview  \
0  [In the 22nd century, a paraplegic Marine is d...   
1  [Captain Barbossa, long believed to be dead, h...   
2  [A cryptic message from Bond’s past sends him ...   
3  [Following the death of District Attorney Harv...   
4  [John Carter is a war-weary, former military c...   

                                         genres  \
0  [Action, Adventure, Fantasy, ScienceFiction]   
1                  [Adventure, Fantasy, Action]   
2                    [Action, Adventure, Crime]   
3              [Action, Crime, Drama, Thriller]   
4           [Action, Adventure, ScienceFiction]   

                                            keywords  movie_id  \
0  [cul

In [62]:
# Vectorize text
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(joined_table['tags'].tolist())
print(vectors)



<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 355166 stored elements and shape (4809, 83318)>
  Coords	Values
  (0, 33591)	0.017081940410114
  (0, 76024)	0.0258802624355175
  (0, 233)	0.08878636993234645
  (0, 12542)	0.05778881449173745
  (0, 58679)	0.08672922932221834
  (0, 49611)	0.1441387463986336
  (0, 34346)	0.020295232990740153
  (0, 21037)	0.08096556564019626
  (0, 76938)	0.014359779885050005
  (0, 54558)	0.0697586705572943
  (0, 58590)	0.08348330226645509
  (0, 57846)	0.024243888696560023
  (0, 79166)	0.07018449602027456
  (0, 54076)	0.05033556828927179
  (0, 10905)	0.0285410606248332
  (0, 7232)	0.04446777565596339
  (0, 77784)	0.0693483331345882
  (0, 7946)	0.046614110687642055
  (0, 26809)	0.06258278668151346
  (0, 57991)	0.07372350541302318
  (0, 3478)	0.014672176240037103
  (0, 62102)	0.0771691374267384
  (0, 3365)	0.024445443484103477
  (0, 2462)	0.10844404240054148
  (0, 14673)	0.07890842503006815
  :	:
  (4808, 18399)	0.10486656756414338
  (4808, 10293)	

In [ ]:
# Apply machine learning algorithm

# Method 1) Recommend with Unsupervised Learning
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(n_neighbors=5, metric='cosine')
model.fit(vectors)

def recommend(movie):
    if movie not in joined_table["title"].values:
        print("Movie not found! Please enter a valid movie name.")
        print("Available movies:", list(joined_table["title"]))
        return
    
    movie_index = joined_table[joined_table['title'] == movie].index[0]
    distances, indices = model.kneighbors(vectors[movie_index])
    print(indices)

    for i in indices[0][1:]:
        print(joined_table.iloc[i].title)

movie = input('Please type a movie title for which you want recommendations.')
recommend(movie)

# Method 2) Recommend with Supervised Learning
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vectors)

def recommend(movie):
    movie_index = joined_table[joined_table["title"] == movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse = True , key = lambda x: x[1])[1:6]
    
    for i in movie_list:
        print(joined_table.iloc[i[0]].title)

recommend(movie)



[[   0 3607  539  151 1202]]
Apollo 18
Titan A.E.
Beowulf
Predators
Apollo 18
Titan A.E.
Beowulf
Predators
Jarhead
